In [1]:
import pandas as pd
import os
import pickle
import time
from pathlib import Path
from datetime import datetime, timedelta

In [2]:
user_entry_time = '09:20:59'
user_exit_time = '15:20:59'
stop_loss_p = 10
max_entries = 5
start_date = datetime.strptime("2022-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2022-12-31", "%Y-%m-%d").date()
market_opening_time = '09:15:59'
market_closing_time = '15:29:59'

In [3]:
df = pd.read_pickle(r"C:\Data\Dataset\Dict\NIFTY\Option_Data\NIFTY_2023.pkl")

In [4]:
instrument = ("NIFTY").upper()

start_date = datetime.strptime("2022-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2022-12-31", "%Y-%m-%d").date()

data_root_path = Path(r"C:\Data\Dataset\Dict")
output_csv_path = Path(r"C:\My_Code\Positional\Sell Positional\Call Sell Positional\Outputs")
# param_file_path = Path(r"C:\My_Code\Positional\Sell Positional\Call Sell Positional\Params\Call_Sell_Positional_Params.csv")

# param_data = pd.read_csv(param_file_path)

# param_data

In [5]:
# dates_range = [str(start_date+tim
                   
                   
#                    edelta(days=x)) for x in range((end_date-start_date).days + 1)]
# year_data = str(start_date.year)

In [6]:
def Filename(year, Instrument):
    year = year
    if Instrument == "BANKNIFTY":
        filename = f"BN_{year}.pkl"
    else:
        filename = f"{Instrument}_{year}.pkl"
    return filename, year

In [7]:
expiry_list = list(df.keys())
expiry_list

['2023-01-12', '2023-01-19', '2023-01-25', '2023-02-02', '2023-02-09']

In [8]:
entry_count = 1

In [11]:
for expiry in expiry_list:
    sl_type = ""
    for n in range(max_entries):
        date_list = list(df[expiry].keys())
        if (sl_type == 'Hit') or (sl_type == ''):
            if sl_type == 'Hit':
                entry_time = exit_time
                date = date_list[date_list.index(exit_date):]
                print("date:", date)
            else:
                entry_time = user_entry_time
                date = date_list
            entry_date = date[0]
            entry_weekday = (pd.Timestamp(entry_date)).day_name()
            print(expiry)
            print("entry_time: ", entry_time, "entry_date : ", entry_date, " entry_weekday : ", entry_weekday)
            if entry_time in list(df[expiry][entry_date].keys()):
                ce_strk_set = list((df[expiry][entry_date][entry_time]["CE"].keys()))
                pe_strk_set = list((df[expiry][entry_date][entry_time]["PE"].keys()))
                common_strike = list(set(ce_strk_set) & set(pe_strk_set))
                price_diff, strikes_list = [], []
                for strk in common_strike:
                    price_diff.append(abs((df[expiry][entry_date][entry_time]["CE"][strk]['close'])-(df[expiry][entry_date][entry_time]["PE"][strk]['close'])))
                    strikes_list.append(strk)
                ATM_strike = strikes_list[price_diff.index(min(price_diff))]
                ep_ce = df[expiry][entry_date][entry_time]["CE"][ATM_strike]['close']
                ep_pe = df[expiry][entry_date][entry_time]["PE"][ATM_strike]['close']
                ep_straddle = ep_ce + ep_pe
                SL = ep_straddle + (ep_straddle * stop_loss_p)/100
#                 for d in date:
#                     if (d == entry_date) & ((pd.Timestamp(entry_date)).day_name() == "Thursday"):
#                         time_list = sorted([key for key, value in df[expiry][d].items() if user_entry_time < key <= user_exit_time])
#                     elif d == entry_date:
#                         time_list = sorted([key for key, value in df[expiry][d].items() if user_entry_time < key <= market_closing_time])             
#                     elif d == last_date_of_week:
#                         time_list = sorted([key for key, value in df[expiry][d].items() if market_opening_time < key <= user_exit_time])
#                     else:
#                         time_list = sorted([key for key, value in df[expiry][d].items() if market_opening_time < key <= market_closing_time])
                    
                for time in time_list:
                    try:
                        straddle = df[expiry][d][time]["CE"][ATM_strike]['close'] + df[expiry][d][time]["PE"][ATM_strike]['close']
                        if(SL <= straddle):  
                            sl_type = "Hit"
                            exit_time = time
                            exit_straddle = straddle
                            CE_PNL = ep_ce - df[expiry][d][time]["CE"][ATM_strike]['close']
                            PE_PNL = ep_pe - df[expiry][d][time]["PE"][ATM_strike]['close']
                            exit_date = d
                            exit_weekday = (pd.Timestamp(exit_date)).day_name()
                            blank = ""
                            break
                        else:
                            sl_type = "Not Hit"
                            blank = 0
                            exit_straddle = straddle
                            exit_time = time
                            CE_PNL = ep_ce - df[expiry][d][time]["CE"][ATM_strike]['close']
                            PE_PNL = ep_pe - df[expiry][d][time]["PE"][ATM_strike]['close'] 
                            exit_date = d
                            exit_weekday = (pd.Timestamp(exit_date)).day_name()
                            if sl_type == "Hit" :
                                entry_date = exit_date
                                entry_weekday = (pd.Timestamp(entry_date)).day_name()

                    except:
                        pass
                print(sl_type)
                print("SL:", SL, " exit_straddle : ", exit_straddle)
                print("exit_date : ", exit_date, "exit_weekday : ", exit_weekday)
                print("exit_time:", exit_time)
                print("CE_PNL:",CE_PNL)
                print("PE_PNL:",PE_PNL) 
                print("CE_PNL + PE_PNL :",CE_PNL + PE_PNL)
            else:
                print(entry_time, "NA")
        else:
            print("DONEZ")
            pass
        print("\n") 

2023-01-12
entry_time:  09:20:59 entry_date :  2023-01-06  entry_weekday :  Friday
Not Hit
SL: 263.0649932861328  exit_straddle :  182.5
exit_date :  2023-01-09 exit_weekday :  Monday
exit_time: 15:29:59
CE_PNL: -15.800003
PE_PNL: 72.45
CE_PNL + PE_PNL : 56.649994


DONEZ


DONEZ


DONEZ


DONEZ


2023-01-19
entry_time:  09:20:59 entry_date :  2023-01-13  entry_weekday :  Friday

SL: 300.85  exit_straddle :  182.5
exit_date :  2023-01-09 exit_weekday :  Monday
exit_time: 15:29:59
CE_PNL: -15.800003
PE_PNL: 72.45
CE_PNL + PE_PNL : 56.649994


2023-01-19
entry_time:  09:20:59 entry_date :  2023-01-13  entry_weekday :  Friday

SL: 300.85  exit_straddle :  182.5
exit_date :  2023-01-09 exit_weekday :  Monday
exit_time: 15:29:59
CE_PNL: -15.800003
PE_PNL: 72.45
CE_PNL + PE_PNL : 56.649994


2023-01-19
entry_time:  09:20:59 entry_date :  2023-01-13  entry_weekday :  Friday

SL: 300.85  exit_straddle :  182.5
exit_date :  2023-01-09 exit_weekday :  Monday
exit_time: 15:29:59
CE_PNL: -15.80000